Purpose of this notebook is to get full text articles from PMC

In [11]:
# imports
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import re
import requests

# import nltk
# from nltk.stem import WordNetLemmatizer
# nltk.download('wordnet')

#from spellchecker import SpellChecker

In [8]:
# reading in the publication list
excel_file = 'data/bkg registry nelists.xlsx'
pubs = pd.read_excel(excel_file, sheet_name='Publication')
# check
pubs.head()

,name,doi,arxiv,pmid,publicationDate,title,authors,abstract,keywords,meshTerms
0,Sun2014,10.1039/C4IB00122B,NaN,25133803.0,2014-08-18,The integrated disease network,Kai Sun and Natalie Buchan and Chris Larminie ...,"The growing body of transcriptomic, proteomic,...","""phenotype, crohn's disease, heterogeneity, ge...","['Computational Biology / methods', 'Databases..."
1,Ernst2015,10.1186/s12859-015-0549-5,NaN,25971816.0,2015-05-14,KnowLife: a versatile approach for constructin...,Patrick Ernst and Amy Siu and Gerhard Weikum,Background: Biomedical knowledge bases (KB's) ...,"['Biomedical text mining','knowledge base','re...","['Biomedical Research', 'Humans', 'Information..."
2,Himmelstein2015,10.1371/journal.pcbi.1004259,NaN,26158728.0,2015-07-09,Heterogeneous Network Edge Prediction: A Data ...,Daniel Himmelstein and Serio Baranzini,The first decade of Genome Wide Association St...,NaN,Algorithms\nAnimals\nChromosome Mapping / meth...
3,Himmelstein2017,10.7554/eLife.26726,10.1101/087619v3,28936969.0,2017-09-22,Systematic integration of biomedical knowledge...,Daniel Scott Himmelstein and Antoine Lizee and...,The ability to computationally predict whether...,NaN,Computational Biology / methods*\nDrug Discove...
4,Martinez2015,10.1016/j.artmed.2014.11.003,NaN,25704113.0,2015-01-13,DrugNet: Network-based drug–disease prioritiza...,Víctor Martínez and Carmen Navarro and Carlos ...,Objective: Computational drug repositioning ca...,Data integration; Disease networks; Drug repos...,Area Under Curve\nComputational Biology*\nComp...


In [9]:
pubs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87 entries, 0 to 86
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   name             87 non-null     object        
 1   doi              86 non-null     object        
 2   arxiv            11 non-null     object        
 3   pmid             76 non-null     float64       
 4   publicationDate  87 non-null     datetime64[ns]
 5   title            87 non-null     object        
 6   authors          86 non-null     object        
 7   abstract         87 non-null     object        
 8   keywords         45 non-null     object        
 9   meshTerms        65 non-null     object        
dtypes: datetime64[ns](1), float64(1), object(8)
memory usage: 6.9+ KB


testing out pubtator

In [29]:
def annotate_pmids(pmids):
    # Define the PubTator API URL
    pubtator_url = "https://www.ncbi.nlm.nih.gov/research/pubtator-api/publications/export/biocjson"

    # Join the PMIDs into a comma-separated string
    pmid_str = ','.join(pmids)

    # Send a GET request to the API with the list of PMIDs
    response = requests.get(pubtator_url, params={"pmids": pmid_str, "full": True})

    # Check if the request was successful
    if response.status_code == 200:
        return response.json()  # Return the response in JSON format
    else:
        raise Exception(f"Error {response.status_code}: {response.text}")

# Example usage
pmids = ["25133803", "25971816"]
annotations = annotate_pmids(pmids)

# Display the annotated results
import pprint
pprint.pprint(annotations)


{'PubTator3': [{'_id': '25971816|PMC4448285',
                'authors': ['Ernst P', 'Siu A', 'Weikum G'],
                'date': '2015-05-14T00:00:00Z',
                'id': '4448285',
                'infons': {},
                'journal': 'BMC Bioinformatics',
                'meta': {},
                'passages': [{'annotations': [],
                              'infons': {'article-id_doi': '10.1186/s12859-015-0549-5',
                                         'article-id_pmc': '4448285',
                                         'article-id_pmid': '25971816',
                                         'article-id_publisher-id': '549',
                                         'elocation-id': '157',
                                         'kwd': 'Biomedical text mining '
                                                'Knowledge base Relation '
                                                'extraction',
                                         'license': 'This is an Open Access 